# Assignment 2

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

An NOAA dataset has been stored in the file `data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv`. The data for this assignment comes from a subset of The National Centers for Environmental Information (NCEI) [Daily Global Historical Climatology Network](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt) (GHCN-Daily). The GHCN-Daily is comprised of daily climate records from thousands of land surface stations across the globe.

Each row in the assignment datafile corresponds to a single observation.

The following variables are provided to you:

* **id** : station identification code
* **date** : date in YYYY-MM-DD format (e.g. 2012-01-24 = January 24, 2012)
* **element** : indicator of element type
    * TMAX : Maximum temperature (tenths of degrees C)
    * TMIN : Minimum temperature (tenths of degrees C)
* **value** : data value for element (tenths of degrees C)

For this assignment, you must:

1. Read the documentation and familiarize yourself with the dataset, then write some python code which returns a line graph of the record high and record low temperatures by day of the year over the period 2005-2014. The area between the record high and record low temperatures for each day should be shaded.
2. Overlay a scatter of the 2015 data for any points (highs and lows) for which the ten year record (2005-2014) record high or record low was broken in 2015.
3. Watch out for leap days (i.e. February 29th), it is reasonable to remove these points from the dataset for the purpose of this visualization.
4. Make the visual nice! Leverage principles from the first module in this course when developing your solution. Consider issues such as legends, labels, and chart junk.

The data you have been given is near **Ann Arbor, Michigan, United States**, and the stations the data comes from are shown on the map below.

In [1]:
import matplotlib.pyplot as plt
import mplleaflet
import pandas as pd
import numpy as np

def leaflet_plot_stations(binsize, hashid):

    df = pd.read_csv('data/C2A2_data/BinSize_d{}.csv'.format(binsize))

    station_locations_by_hash = df[df['hash'] == hashid]

    lons = station_locations_by_hash['LONGITUDE'].tolist()
    lats = station_locations_by_hash['LATITUDE'].tolist()

    plt.figure(figsize=(8,8))

    plt.scatter(lons, lats, c='r', alpha=0.7, s=200)

    return mplleaflet.display()

leaflet_plot_stations(400,'fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89')

FileNotFoundError: File b'data/C2A2_data/BinSize_d400.csv' does not exist

In [ ]:
from matplotlib.ticker import NullFormatter
from matplotlib.dates import MonthLocator, DateFormatter

df = pd.read_csv('data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv')
df.set_index("ID", inplace=True)
df_UMich = df.loc['USC00200230'].sort(['Date'])
df_UMich = df_UMich.reset_index()

# Here, let's use only data from 2005 to 2014, not 2015, and re-name the dataframe as df_UMich
df_UMich_2005_2014 = df_UMich.loc[df_UMich['Date'].apply(lambda x: x.split('-')[0] in ['2005','2006','2007','2008','2009','2010','2011','2012','2013','2014'] )] 

df_UMich_TMAX_2005_2014 = df_UMich_2005_2014.loc[df_UMich['Element'] == 'TMAX']
df_UMich_TMIN_2005_2014 = df_UMich_2005_2014.loc[df_UMich['Element'] == 'TMIN']

# list of all days without year
days_list = []
for date in df_UMich_2005_2014['Date']:
    if '2008' in date:
        days_list.append(date)

days_without_year_list = []
for date in days_list:
    days_without_year_list.append(date[5:])

# Next, get record high and record low of each date between 2005 to 2014
date_TMAX_dic_2005_2014 = {}
    
date_TMAX_dic_2005_2014 = {day_el: [] for day_el in days_without_year_list}

for day_el in days_without_year_list:
    for el_df_UMich_TMAX_2005_2014 in df_UMich_TMAX_2005_2014['Date']:
        if day_el in el_df_UMich_TMAX_2005_2014:
            date_TMAX_dic_2005_2014[day_el].append(df_UMich_TMAX_2005_2014.loc[df_UMich_TMAX_2005_2014['Date'] == el_df_UMich_TMAX_2005_2014]['Data_Value'])
            

df_dic = date_TMAX_dic_2005_2014
date_TMAX_dic_2005_2014_2 = {}
date_TMAX_dic_2005_2014_2 = {day_el: [-500] for day_el in days_without_year_list}

for day_dic_el in df_dic:
    for el in df_dic[day_dic_el]:
        if el.values[0] > date_TMAX_dic_2005_2014_2[day_dic_el]:
            date_TMAX_dic_2005_2014_2[day_dic_el] = el.values[0]

date_TMAX_dic_2005_2014_2.pop('02-29')

#Now I got max temp and min temp on each day
date_TMIN_dic_2005_2014 = {}
    
date_TMIN_dic_2005_2014 = {day_el: [] for day_el in days_without_year_list}

for day_el in days_without_year_list:
    for el_df_UMich_TMIN_2005_2014 in df_UMich_TMIN_2005_2014['Date']:
        if day_el in el_df_UMich_TMIN_2005_2014:
            date_TMIN_dic_2005_2014[day_el].append(df_UMich_TMIN_2005_2014.loc[df_UMich_TMIN_2005_2014['Date'] == el_df_UMich_TMIN_2005_2014]['Data_Value'])
            
df_dic = date_TMIN_dic_2005_2014
date_TMIN_dic_2005_2014_2 = {}
date_TMIN_dic_2005_2014_2 = {day_el: [500] for day_el in days_without_year_list}

for day_dic_el in df_dic:
    for el in df_dic[day_dic_el]:
        if el.values[0] < date_TMIN_dic_2005_2014_2[day_dic_el]:
            date_TMIN_dic_2005_2014_2[day_dic_el] = el.values[0]

date_TMIN_dic_2005_2014_2.pop('02-29')

# do the same thing with only 2015 data

df_UMich_2015 = df_UMich.loc[df_UMich['Date'].apply(lambda x: x.split('-')[0] == '2015')]
df_UMich_TMAX_2015 = df_UMich_2015.loc[df_UMich_2015['Element'] == 'TMAX']
df_UMich_TMIN_2015 = df_UMich_2015.loc[df_UMich_2015['Element'] == 'TMIN']

date_TMAX_dic_2015 = {}
    
date_TMAX_dic_2015 = {day_el: [] for day_el in days_without_year_list}

for day_el in days_without_year_list:
    for el_df_UMich_TMAX_2015 in df_UMich_TMAX_2015['Date']:
        if day_el in el_df_UMich_TMAX_2015:
            date_TMAX_dic_2015[day_el].append(df_UMich_TMAX_2015.loc[df_UMich_TMAX_2015['Date'] == el_df_UMich_TMAX_2015]['Data_Value'])
            
df_dic = date_TMAX_dic_2015
date_TMAX_dic_2015_2 = {}
date_TMAX_dic_2015_2 = {day_el: [-500] for day_el in days_without_year_list}

for day_dic_el in df_dic:
    for el in df_dic[day_dic_el]:
        if el.values[0] > date_TMAX_dic_2015_2[day_dic_el]:
            date_TMAX_dic_2015_2[day_dic_el] = el.values[0]

date_TMAX_dic_2015_2.pop('02-29')

date_TMIN_dic_2015 = {}
    
date_TMIN_dic_2015 = {day_el: [] for day_el in days_without_year_list}

for day_el in days_without_year_list:
    for el_df_UMich_TMIN_2015 in df_UMich_TMIN_2015['Date']:
        if day_el in el_df_UMich_TMIN_2015:
            date_TMIN_dic_2015[day_el].append(df_UMich_TMIN_2015.loc[df_UMich_TMIN_2015['Date'] == el_df_UMich_TMIN_2015]['Data_Value'])
            

df_dic = date_TMIN_dic_2015
date_TMIN_dic_2015_2 = {}
date_TMIN_dic_2015_2 = {day_el: [-500] for day_el in days_without_year_list}

for day_dic_el in df_dic:
    for el in df_dic[day_dic_el]:
        if el.values[0] > date_TMIN_dic_2015_2[day_dic_el]:
            date_TMIN_dic_2015_2[day_dic_el] = el.values[0]

date_TMIN_dic_2015_2.pop('02-29')


import matplotlib.pyplot as plt
one_year_array = pd.Series(range(1,len(date_TMAX_dic_2005_2014_2)+1)).values


TMAX_df_from_dic_to_array = pd.DataFrame.from_dict(date_TMAX_dic_2005_2014_2, orient='index').values
TMIN_df_from_dic_to_array = pd.DataFrame.from_dict(date_TMIN_dic_2005_2014_2, orient='index').values

# convert an array to a list and divided by ten, then convert back to an array from the list
TMAX_df_from_dic_to_array_to_list_of_list = TMAX_df_from_dic_to_array.tolist()
TMAX_df_from_dic_to_array_to_list = []
for el in TMAX_df_from_dic_to_array_to_list_of_list:
    TMAX_df_from_dic_to_array_to_list.append(el[0])
TMAX_df_from_dic_to_array_from_list = np.array([x/10 for x in TMAX_df_from_dic_to_array_to_list])

TMIN_df_from_dic_to_array_to_list_of_list = TMIN_df_from_dic_to_array.tolist()
TMIN_df_from_dic_to_array_to_list = []
for el in TMIN_df_from_dic_to_array_to_list_of_list:
    TMIN_df_from_dic_to_array_to_list.append(el[0])
TMIN_df_from_dic_to_array_from_list = np.array([x/10 for x in TMIN_df_from_dic_to_array_to_list])

# Next, find dates when 2015 broke the record high and low bewteen 2005 and 2014
count_1 = 1
TMAX_broke_day_index_list = []
TMAX_borke_temp_list = []
for el_1,el_2 in date_TMAX_dic_2015_2.items():
    if el_2 > date_TMAX_dic_2005_2014_2.get(el_1):
        TMAX_broke_day_index_list.append(count_1)
        TMAX_borke_temp_list.append(el_2)
        count_1 += 1
    else:
        count_1 += 1

count_2 = 1
TMIN_broke_day_index_list = []
TMIN_borke_temp_list = []
for el_1,el_2 in date_TMIN_dic_2015_2.items():
    if el_2 < date_TMIN_dic_2005_2014_2.get(el_1):
        TMIN_broke_day_index_list.append(count_2)
        TMIN_borke_temp_list.append(el_2)
        count_2 += 1
    else:
        count_2 += 1

TMAX_broke_day_array = np.array(TMAX_broke_day_index_list)
TMIN_broke_day_array = np.array(TMIN_broke_day_index_list)
TMAX_broke_temp_array = np.array([x/10 for x in TMAX_borke_temp_list])
TMIN_broke_temp_array = np.array([x/10 for x in TMIN_borke_temp_list])


plt.figure()

observation_dates = np.arange('2015-01-01', '2016-01-01', dtype = 'datetime64[D]')

count_3 = 1
TMAX_broke_obs_date_list = []
for obs_dates_el in observation_dates.tolist():
    if count_3 in TMAX_broke_day_index_list:
        TMAX_broke_obs_date_list.append(obs_dates_el)
        count_3 += 1
    else:
        count_3 += 1
TMAX_broke_obs_date_array = np.array(TMAX_broke_obs_date_list)


count_4 = 1
TMIN_broke_obs_date_list = []
for obs_dates_el in observation_dates.tolist():
    if count_4 in TMIN_broke_day_index_list:
        TMIN_broke_obs_date_list.append(obs_dates_el)
        count_4 += 1
    else:
        count_4 += 1
TMIN_broke_obs_date_array = np.array(TMIN_broke_obs_date_list)    
    
plt.plot(observation_dates, TMAX_df_from_dic_to_array_from_list, 'tab:orange', zorder = 1, 
         label = '10-year (2005-2014) Record High')
plt.plot(observation_dates, TMIN_df_from_dic_to_array_from_list, 'xkcd:lightblue', zorder = 1,
         label = '10-year (2005-2014) Record Low')
plt.plot(TMAX_broke_obs_date_array, TMAX_broke_temp_array, 'ro', zorder = 1, 
         label = '2015 New High')
plt.plot(TMIN_broke_obs_date_array, TMIN_broke_temp_array, 'bo', zorder = 1, 
         label = '2015 New Low')

ax = plt.gca()
# Set title
from textwrap import wrap
ax.set_title("\n".join(wrap('Record High and Low Temperatures near Ann Arbor by Day',35)))
# Set labels for the y axes.
ax.set_ylabel('$^\circ C$', rotation = 0, position = (0, 0.94))

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)

# Shade the area between the area between the record high and record low temperatures.
ax.fill_between(x = observation_dates, 
                y1 = TMAX_df_from_dic_to_array_from_list, y2 = TMIN_df_from_dic_to_array_from_list, 
                facecolor = 'lavenderblush')

# Add the 0 degree celsius line.
plt.axhline(linestyle = '--', color = 'gray', linewidth = 0.3) 

################ Deal with the legend, axis, title, etc. ##################
plt.legend(bbox_to_anchor = (1.8, 0.5), frameon = False)

# Increase the left and right margin to better show the edge points.
ax.set_xlim(observation_dates[0] - 10, observation_dates[364] + 10)

# Set major x ticks at the beginning/end of each month and add minor ticks at the 
# 15th day of each month.
ax.xaxis.set_major_locator(MonthLocator())
ax.xaxis.set_minor_locator(MonthLocator(bymonthday = 15))

# Put the abbr. month labels in the middle of of the span of each month.
ax.xaxis.set_major_formatter(NullFormatter())
ax.xaxis.set_minor_formatter(DateFormatter('%b'))

plt.show()